In [1]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get("http://showlistaustin.com/")

In [4]:
events = driver.find_elements_by_css_selector("td")

In [5]:
import re

In [61]:
# regex to remove words between []
x = events[1].text
new_word = re.sub("[\[].*?[\]]", "", x)
# split on \n. output list
word_list = new_word.splitlines()
# loop. remove spaces at end of text, regex to remove last parenthesis text (in this case the address)
for word in word_list:
    print(re.sub("\([^()]*\)$", "", word.rstrip()))

Khruangbin, The Mattson 2 at the Mohawk 
Palm Daze, Wonderbitch, Chakra Khan at the Mohawk 
Versus Cult: Sound & Fury Edition with Art Acevedo, Blank Hellscape, Bucket of Piss, Pfisters, Jarebear, Homo-Erectus at Beerland 
Altamesa Single Release Show with JD Clark, Jesse Ebaugh, & The Tender Things at Hotel Vegas 
Rosette (Houston), Lowin, Hey Jellie (members of Erase Errata), and The Naked Tungs at Barracuda 
Human Influence Presents Raise Her Voice with Treelah, amea, Darja, Violinda Lola, Tenci, Niko El Narco, Helen Hailu, DJ Cateye at Cheer Up Charlies 
Heat Wave presented by Brodie's Fault with Night Cap, The Brook & The Bluff, Teqeela Mockingbirds at the Spiderhouse Ballroom 
Random Rab at Empire Control Room 
Cut Copy at Stubb's Waller Creek Amphitheater 
Electric Six, Northern Faces at Stubb's 
Kelly Hafner (record release) at Flamingo Cantina 
Hirsute, Ziggurat, vinylboss, Savannah Red at Dozen Street 
Jeff Plankenhorn at Oskar Blues Brewery 
Magnifico (Queen tribute), Queue 